## Beyond Single-species Systems
So far, we have only seen dislocations of single-species systems. However, the above dislocation classes can also be applied to more complex ordered systems, under the following constraints:
1. The desired system can be expressed in a cubic lattice
2. The desired system shares geometry with an existing crystal structure

As an example, let's take Zincblende InP and compare with a diamond lattice:

In [ ]:
from ase.build import bulk
from ase.lattice.cubic import Diamond
import numpy as np

# Data from https://pubs.acs.org/doi/epdf/10.1021/acs.jpca.0c02450 DFT values
alat = 5.84
C11 = 99.3
C12 = 55.4
C44 = 45.0

InP = bulk("InP", crystalstructure="zincblende", cubic=True, a=alat)

diamond = Diamond("C", latticeconstant=alat)

inp_pos = InP.get_scaled_positions()
dia_pos = diamond.get_scaled_positions()

# Sort the coords, as bulk uses a different atom order
inp_idxs = np.lexsort((inp_pos[:, 0], inp_pos[:, 1], inp_pos[:, 2]))
dia_idxs = np.lexsort((dia_pos[:, 0], dia_pos[:, 1], dia_pos[:, 2]))
print("InP Fractional Coordinates")
print(inp_pos[inp_idxs, :])
print()
print("Diamond Fractional Coordinates")
print(dia_pos[dia_idxs, :])

interactive_view(InP, scale=0.4)

We can see that the fractional coordinates of the Zincblende InP are the same as the Diamond structure we generated - this is what was meant by the second constraint. The equivalent geometry means that the continuum-scale displacement field used to build the dislocations will be applied to atoms in the same fractional coordinates, allowing us to get an equivalent core reconstruction.

To actually build a dislocation with this bulk, we can simply pass it as an argument, in place of the lattice constant:

In [ ]:
from matscipy.dislocation import DiamondGlide60Degree

disloc = DiamondGlide60Degree(InP, C11, C12, C44)

InP_bulk, InP_dislocation = disloc.build_cylinder(radius=20)

interactive_view(InP_dislocation, scale=0.4)

### Dislocations in Disordered Systems
Generating dislocation structures for systems which have some chemical disorder is also possible, with some caveats:
1. The generation routines will only work reliably when given on-lattice bulk structures with cubic cells
2. The generated structure is based on a periodic replication of the base unit_cell, thus won't have true disorder
3. The displacement field applied to the bulk structure does not depend on atomic species - known lattice distortions caused by the disorder will not be captured
4. The current code only allows $C_{11}$, $C_{12}$, and $C_{44}$ to be specified, so more complex elastic effects (E.G. $C_{11} \neq C_{22}$) cannot currently be captured

With these in mind, a recommended workflow would be to start by generating the dislocation system for an ordered system, but using the elastic constants of the disordered system. From there, the disorder can be applied by simply changing the chemcical symbols of the atoms to match the target composition.

To show a worked example of this, consider the alloy $\text{In}_{0.5} \text{Ga}_{0.5} \text{As}$. This should form a distorted Zincblende structure, where the In sites from the previous InP bulk are now 50% occupied by Ga, and all P sites are now In.

In order to model this, we can generate a dislocation for GaAs (which is very similar to the InP example above), but using the lattice constant and elastic properties of $\text{In}_{0.5} \text{Ga}_{0.5} \text{As}$.

In [ ]:
from ase.build import bulk
from matscipy.dislocation import DiamondGlide60Degree

alat = 5.87 # Rough value from https://www.ioffe.ru/SVA/NSM/Semicond/GaInAs/mechanic.html
# Data from https://doi.org/10.1080/08927022.2011.602975
C11 = 120.31
C12 = 55.87
C44 = 58.26

GaAs = bulk("GaAs", crystalstructure="zincblende", cubic=True, a=alat)


disloc = DiamondGlide60Degree(GaAs, C11, C12, C44)

GaAs_bulk, GaAs_dislocation = disloc.build_cylinder(radius=20)

interactive_view(GaAs_dislocation, scale=0.4)

From this GaAs structure, we then do Monte Carlo sampling to introduce 50% In to the structure:

In [ ]:
import numpy as np

np.random.seed(2)

species = np.array(GaAs_bulk.get_chemical_symbols())

Ga_idxs = np.argwhere(species == "Ga")[:, 0]

# Choose half the idxs to be In
In_idxs = np.random.choice(Ga_idxs, size=int(Ga_idxs.shape[0]/2), replace=False)

# Introduce the chemical disorder in In-Ga sites
species[In_idxs] = "In"

InGaAs_bulk = GaAs_bulk.copy()
InGaAs_bulk.set_chemical_symbols(species)

InGaAs_dislocation = GaAs_dislocation.copy()
InGaAs_dislocation.set_chemical_symbols(species)

interactive_view(InGaAs_dislocation, scale=0.4)